<a id=index></a>
# Index
- <a href=#s3>S3</a>
- <a href=#cloudfront>CloudFront</a>
- <a href=#ec2>EC2</a>

<a id=s3></a>
## S3
<a href=#index>Go to Index</a>

### Generic info
- Object based storage (not block storage)
- ACL are used to control access to specific resources. Bucket policy applies to the whole bucket.
- When a file is uploaded to S3, the browser gets a 200 response for a successful upload. 
- All objects are private by default
- Min sized object on S3 is 0 bytes (eg: Delete marker)
- 3 types of storage - Standard, IA (Infrequently accessed), Reduced Redundancy Storage
- Can use client side or server side encryption. Server side encryption consists of 3 options -
	- S3 (SSE-S3)
	- KMS (SSE-KMS)
	- Customer provided keys (SSE-C)
- S3 Standard: 11 9s of durability and 4 9s of availability and can stand the loss of two facilities concurrently
- S3 IA: Data that is infrequently retrieved but needs to be accessible quickly when needed. Cheaper than Standard but charged per retrieval
- 'S3 One Zone - Infrequently Accessed' is replacing Reduced Redundancy Storage (RRS)
    - For data that need not be stored in multiple AZ and is infrequently accessed
    - Availability is 99.5%
- Glacier: Is for data archival. 3 types of retrieval (decreasing order of cost, increasing order of time)
    - Expedited
    - Standard
    - Bulk
- S3 buckets can be configured to create access logs (can dump all logs to another bucket)
- S3 uploads can get much faster if done as multi-part upload
- Can have upto 100 buckets per account by default
- Single operation upload to S3 works for files <5GB. Multi-part works for files from 5MB to 5TB.

### Consistency
- Read after Write consistency for new object PUTS (instantaneous)
- Eventual consistency for overwrite PUTS and DELETE actions (can take a little time to propagate)

### Versioning
- For a public file with versioning enabled, it will be made private in the new version. 
- Versioning cannot be removed (can be suspended)
- Actually when you delete a file, it gets another version called 'delete marker'. If you delete the delete marker, the file gets restored.

- The files only get removed when you got to versioning and remove the versions themselves
- There is also an option to enable MFA for deleting files with versioning enabled.

### Cross Region Replication (CRR)
- When CRR is newly introduced on a bucket, the existing objects are not automatically copied. New and updated objects are copied to the replication bucket. AWS CLI (s3 cp --recursive) can be used to copy all files from the primary to the replication bucket
- Versioning must be enabled (source and target) for CRR
- Src and target should be in different regions (Cross **Region** replication)
- At this time, you cannot do replication to multiple buckets (daisy chaining is not supported at present)
- Deleting files (delete marker) or deletion of individual versions are not replicated to the secondary buckets (this is an AWS security feature

### Lifecycle rule
- Main idea is to reduce cost
- S3 can automatically move contents of a bucket (or folder or things with a spefied tags ) to S3-IA and later to S3 Glacier based on what we set in the Management>Lifecycle tab added as Lifecycle rule. - Can also be configured to permanently delete stuff at some point
- To move to S3-IA requires a minimum of 30 days after creation and moving to Glacier requires 30 days after residing in IA or 60 days from creation
- Can be done along with versioning.

### S3 Security & Encryption
- By default all S3 objects are private
- Use SSL/TLS (https) for encryption during transit
- At rest use one of three server side encryption options 
    - S3 Managed Keys (SSE-S3)
    - S3 KMS (charges you, tells you who is decrypting what when)
    - Customer provided key (SSE-C). You handle the keys
- Can also choose to do client side encryption

### Storage Gateway
Is a service that connects local on-prem IT network with a cloud storage for scalable and cost-effective storage. It can connect to the on-prem storage and automatically replicate things on to S3 or S3 Glacier.

There are 4 types of storage gateways -
1. File Gateway (NFS) - for media files, flat files, etc.. stored directly on S3
2. Volume Gateways (iSCSI):  for block based storage. Think of these as a connection to harddisks(disk **volumes**). Data written to these disk volumes can be backed up using EBS. These can be used to save system snapshots. These snapshots are incremental (only changes get captured). Snapshot storage is also compressed to minimize storage costs.
    - Stored Volumes: Entire dataset is stored on site and is asynchronously backed up to S3
    - Cached Volumes: Entire data is stored on S3 and most frequently accessed data is cached on site
3. Virtual Tape Library Gateway (VTL): Used for backup. Uses popular applications like NetBackup, Backup exec, Veenam etc.
    

### Snowball

Legacy option was Import/Export Disk (portable disks) that Amazon allowed for large scale data transfer. 

Snowball can import to and export from **S3**.
- Snowball: 256 AES encryption, tamper proof, multiple layer security, trusted platform module. comes with a kindle to track the device. Software erase after each transfer (so that no one can retrieve data from it)
- Snowball Edge: like Snowball, but also has compute capacity. Eg: Airline manufacturer can place this device in a test flight plane for data capture. Like an mobile AWS datacenter in a box.
- Snowmobile: Petabytes or exabytes of data. Truck trailer. Even a complete datacenter migration.

### S3 Transfer Acceleration
Instead of uploading drectly to an S3 bucket in some region directly, upload to an edge location that is close to you.

Within S3 Properties>Transfer acceleration. Provides a new endpoint.

### S3 Static Website Hosting
- The public endpoing follows a specific format "<bucket-name>-s3-website-<region>-amazonaws.com"
- Use bucket policy to make entire S3 bucket public
- S3 scales automatically to meet demand for the website


<a id=cloudfront></a>
## CloudFront
<a href=#index>Go to Index</a>

### Generic
- CDN - Content Delivery Network. Something that enables faster content delivery of content if the geographical location of the requestor and requested content might otherwise cause delay in service.
- Edge location (EL) is different from AWS region or AZ. Currently there are over 50 ELs. 
- Edge location takes content from origin. At this time origin can be 
	- S3 bucket
	- EC2 instance
	- Elastic Load Balancer
	- Route53
	- Can also be an external source of the files
- Distribution is the name given to the CDN which consists of a collection of Edge locations
- When content is requested first time, it is fetched from origin and then cached in the EL. This is cached for a specified TTL (Time To Live)
- There are two types of distribution
	- Web Distribution - Typically used for websites
	- RTMP - used for media streaming
- Edge locations are **not read-only**. They can also be used for put operations and that object will then be written back to the origin.
- You can clear the cached objects but you will be charged for it. The normal behavior is that the cached object will expire on its own after the specified TTL.
- Specifics of setting up the CloudFront Distribution
    - CloudFront uses an identity to access the S3 bucket (Origin Access Identity parameter in the CloudFront dist setup)
    - In the setup, remember to 'Grant Read permissions on bucket' to 'yes'
    - Allow HTTP and HTTPS(recommended for regular pages)
    - TTL is an important design consideration (set based on how frequently the objects change)
    - Pick the default SSL certificate so as to support HTTPS unless you can publish a certificate
    - There is a feature called Restrictions - allows Geo-Restrictions using a Whitelist and a Blacklist. Blacklist allows global access and restricts to few entities. Whitelist only serves to the entities in the whitelist.
    - "Invalidation" is the process of removing an object that is cached on CloudFront. Remember that this is to delete objects before TTL. *This costs money.*

<a id="ec2">
# EC2: Elastic Cloud Compute
<a href="#index">Back to Index</a>

Provides resizeable compute capacity in the cloud: 
    - can scale up and down (increase/decrese capacity by starting new instances in minutes)
    - can scale out (Elastic load balancer)
    
EC2 can be provisioned -
- On Demand: 
    - Pay fixed rate by the hour - Windows (or by the second - Linux)
    - Good for temporary needs (Dev)
- Reserved instances: 
    - Contracts with Amazon 1 or 3 year contracts. Significant dicount (75% off of on-demand prices)
    - Great for predictible usage
        - Standard RIs (upto 70% off on-demand)
        - Convertible RIs (upto 54% off on-demand) allow you to change the attributes of the RIs as long as it creates instances of equal or higher value
        - Scheduled RIs: good for a known pattern in demand/requests.
- Spot Instances: 
    - By at a price that you are comfortable with (like a market)
    - Flexible start and end price
    - Applications that are only feasible at low compute prices
    - If Spot instance is terminated by Amazon EC2, you will not be charged for a partial hour of usage. However, if you terminate the instance yourself, you will be charged for the complete hour in which the instance ran.
- Dedicated Hosts: 
    - A physical machine for you. Eg: If your license is attached to a mac id, you can use it on cloud by hosting it in a Dedicated host. 
    - It could be because of regulatory requirements too
    - Can be purchased on-demand and can also be reserved (upto 70% off on-demand price)

Pneumonic to remember all instance types: FIGHT DR.Mc PX. There are instances that star with all these letters. Think Edward Norton fights (Fight) Dr. McDonald (Dr.Mc) who is from Scotland and hands out pictures (PX) of Scotlant

### EBS
Elastic Block Storage. If EC2 are servers online, EBS are disks online. EBS is placed in a specific AZ and replicated within that AZ.
Comes in different kinds - 
    - General purpose SSD (GP2): 
        - balances price and performance
        - speed of 3 to 10k IOPS per GB. And the ability to burst upto 3000 IOPS for extended periods of time for volumes 3334 GB and above
   - Provisioned SSD IOPS:
       - For I/O intensive applications like large relational or NoSQL databases
       - if you need more than 10,000 IOPS
       - Can provision up to 20,000 IOPS
   - Throughput Optimized HDD (ST1) (magnetic disk):
       - Cannot be boot volume. Can only be additional volumes (eg: cannot be C drive in Windows)
       - Good for big data, data warehousing, log processing etc
   - Cold HDD (SC1) (magnetic disk)
       - Lowest cost for infrequently accessed workloads
       - File server
       - Cannot be boot volume
   - Magnetic (Standard) (legacy):
       - Lowest cost per GB of all EBS bootable vol types
       - Ideal for worlkoads where data is accessed infrequently
       - For applications where the lowest storage cost is important


### EC2 Hands-on session
- Tenancy is the property that enables you to use a dedicated host or a shared host
- Remember that one Subnet (within a VPC) falls in one AZ
- Monitoring property during EC2 allows for detailed monitoring (RAM, CPU utilzation etc. every 1 min). Default is to track these every 5 mins
- Delete on Termination is the default behavior for EBS volumens attached to EC2 instances
- There are two checks that are listed on EC2 console
    - System status check: Ensures network connectivity to the instance is working fine. This is equivalent to checking that the Hypervisor level things are working alright
    - Instance status check: Ensures the OS is able to receive packages 
    
- By default, the root volume cannot be encrypted. It can be encrypted by creating a copy and encrypting it and being used as root volume.
- Termination protection is turned off by default
- On EBS backed instance, the default action is for the root EBS volume to be deleted when instance is terminated
- EBS root volumes of your default AMIs cannot be encrypted. Can use a third party tool such as bit locker to encrypt root volumes. 
- Additional EBS volumes can be encrypted
- Terminated instances will be visible in the list of instances for 1 hour after termination
    

### Security Groups
- Any change that is made to an SG takes effect immediately
- SGs are STATEFUL (in contrast with a Network Access Control list). Meaning if you allow inbound rules, then those will also receive outbound traffic even if there are no rules mentioned in the outbound section of the SG.
- You cannot deny access of specific kind or for a specific IP address using SGs. You can only 'allow' using SGs. There are no inbound rules by default
- All outbound traffic is allowed by default
- Can attach multiple SGs to an instance
- Can have any number of EC2 instances within an SG

### EBS volumes
- EBS volumes have to be in the same AZ as the EC2 instance
- To make a copy of EBS in a different AZ, make a snapshot and create volume in required AZ using snapshot
- Use the Actions>Copy option to create the instance in another Region
- Except 'Standard' (primitive magnetic), all other volumes type and sizes can be edited after creation. No associated downtime
- EBS Snapshots are created for backups and EBS images are used to clone EC2 instances
- Root volume is deleted when EC2 instance is terminated. Other EBS volumes are not terminated
- Can take snapshots when instance is running except if you're taking snapshots of EBS volumes that serve as root volumes
- Snapshot of encrypted volumes are encrypted. Volumes restored from encrypted snapshots are encrypted. 
- Only snapshots of unencrypted snapshots can be shared publicly

Volumes exist on EBS. Snapshots exist on S3. Snapshots are incremental (only changes from last snapshot are captured)

### AMI 
AMI is selected based on 
- Region
- OS
- Architecture
- Launch Permissions
- Storage for the Root Device
- Storage for Root Device - Instance Store (Ephemeral Storage) or EBS backed volumes
    - Remember that instance store cannot have 'start'/'stop' but EBS backed has these. When they are started after being stopped, it starts in a different hypervisor
    - Instance store backed instances will lose data if the underlying host fails
    - EBS volumes can be detached from one instance and attached to another. This cannot be done for Instance based volumes
    - Both types can be rebooted without loss of data
    
### Elastic Load Balancer
Come in 3 types - 
- Application load balancer: Very advanced, can send specific request to specific web servers, work at layer 7 of TCP model
- Network load balancer: For extreme performance, works at connection layer (layer 4). Can handle millions of requests per sec with ultra low latency
- Classic load balancer: These are legacy load balancers (what are meant when Elastic Load Balancer is mentioned). Can balance HTTP/HTTPS requests and also use layer-7 specific features (X-forwarded-For and sticky sessions). Can also use strict layer 4 LB for applications that rely strictly on TCP protocol. 

X-Forwarded-For header can be used to pass the public ip (IPv4) of the requester to the EC2 instance from the ELB

If the application stops responding ELB responds with Error code 504 (Gateway Timed Out). Could be because the Web Server or the Database Server failed

ELBs do healthchecks for EC2 instances and keeps track of "InService" or "OutOfService"

### Cloudwatch
Can be used to monitor components and also set alarms to notify about events of interest
- Standard monitoring: 5 mins
- Detailed monitoring: 1 mins

**Remember that Cloudwatch is for monitoring and CloudTrail is for audit. CloudTrail captures everything that happens within the AWS account**

**EC2 bash script (on startup)**
 - If aws s3 command is used for buckets and EC2 is separate regions, specify the --region parameter with the region of the EC2 instance

**EC2 Instance Metadata**
- Can get instance metadata at http://169.254.169.254/latest/meta-data/ once a user ssh's into the EC2 instance. Use curl command to get response
- curl http://169.254.169.254/latest/user-data/ to get the bash script provided to the EC2 instance (it is called user-data)

### Auto Scaling Groups

- Needs a Launch configuration that runs when new EC2 instances are created
- Can choose to increase/decrease instances based on criteria set by user
- If a set number of instances are chosen, it creates instances when some of them goes down


#### Placement Groups (PG)
- Name of the PG must be unique within an AWS a/c
- Only certain types of instances can be launched in PGs (Compute Optimized, GPU, Memory Optimized, Storage Optimized). 
- AWS recommends homogeneous instances within placement groups
- Cannot merge PGs
- Cannot move an existing instance into a PG
- Two types - 
    - Clustered Placement Group (is the older and default one): **All instances are in a single AZ.** When low-network latency is necessary. Eg: if you want to manually set up Hadoop cluster. 
    - Spread Placement Group: For a group of instances that are placed in distinct underlying hardware. Good for cases where a small number of critical instances are to be kept separate from each other. Good to keep instances in multiple AZs (kind of the opposite of Clustered PG)

### Elastic File Storage
- Block based storage
- File storage service for EC2
- Storage capacity is elastic, increases and decreases based on more files being added or deleted
- Supports NFS v4 protocol (Network File System version 4)
- No pre-provisioning required, only pay for the storage you use (30 cents per GB)
- Can scale to Petabytes
- Can support 100s of concurrent NFS connections
- Data is stored across multiple AZ's within a region
- Read after write consistency

#### Lambda
- Automatically scales out
- Function invocations are independent (1 event = 1 function invocation)
- No servers!
- Super cheap!
- Example, Amazon Alexa is powered entirely by Lambda functions
- Function should end within 5 mins
- Triggers
    - API Gateway
    - S3
    - SNS
    - Kinesis
    - DynamoDB
    - AWS IoT
    - Alexa Skills Kit
    - Alexa Smart Home
- Languages
    - .net (C#/Powershell)
    - Go
    - Java
    - Python (2 & 3)
    - Ruby
    - Custom runtime
- Billing
    - Number of requests
    - First 1 million reqs are free; \$0.2 per 1 million requests 
    - Duration of code execution rounded up to 100th mS. 
    - Price depends on amt of memory used too. $0.00001667 per GB-second used
    
- AWS X-Ray can be used to debug if multiple components are connected and something goes wrong

### Read up on
- Kubernitics for setting up Docker container EC2 cluster
- RAID for backup of EC2 storage devices
- EC2 hypervisors
    - Nitro
    - Xen

## DNS
- Route 53 was named for the highway that goes across US and because DNS is on port 53
- Top level domains are controlled by Internet Assigned Numbers Authority (IANA)
- Registrars are entities that have permission to allot domain names that map directly to top level domain names eg: Amazon, GoDaddy etc.
- The domain names alloted by registrars are registered with InterNIC, a service of ICANN. Each domain name becomes registered in a central database known as the WhoIS database
- NS record is Name Server records. Used by top level Domain servers to direct traffic to the Content DNS server which contains the authoritative DNS records
- Start of Authority Record (SOA)
    - name of server that supplied the data for the zone
    - The administrator of the zone
    - The current version of the data file
    - Default number of seconds for the time-to-live file on resource records
- "A" Record is a fundamental DNS record to translate a domain name to an ip address
- CName (Canonical Name) can be used to resolve one domain name to another
- Alias Records is very much like CName in that it is used to map one domain name to another. 
    - Difference between CName and Alias Records is that Alias records can handle Naked domain names and CName cannot
    - Naked domain names are ones without 'www' preceding the domain name. eg: http://acloud.guru
    - Alias Records are more capable and hence to be chosen if given an option b/w CNames and Alias Records
    - Route 53 recognizes changes in record sets the alias records refers to
- MX records, PTR records are also relatively common DNS jargon
            
- Remember that ELBs do not have a pre-defined IP address, you resolve them using a DNS name
- DNS Zone is the part of a domain for which an individual DNS server is responsible
- SOA (Start of Authority) record is information stored in a DNS zone about that zone


## Route 53
- Important to know the different routing policies 
    - Simple: Can only create one record but that record can have mapping to multiple IPs and it allots each request randomly 
    - Weighted: Can allot some 30% traffic to one AZ, 20% traffic to another AZ and the rest somewhere else.
    - Latency: Route requests to server with minimal latency for user
    - Failover: Have to set up healthcheck. There should be Primary and Secondary IPs provided
    - Geolocation: Routes requests based on the Geo source of requests. 
    - Multivalue Answer: Kind of like Simple routing but can have different A records pointing to different IP addresses. Can have 12 IP addresses and upto 8 health checks and it will 

#### Read more
- https://docs.aws.amazon.com/Route53/latest/DeveloperGuide/ResourceRecordTypes.html#MXFormat


## Databases
#### Relational
- SQL Server
- Oracla
- MySQL
- PostgreSQL
- Aurora
- MariaDB
#### Non-relational
Has Collection (Table), Document (Row), Key-Value pairs (Fields)
#### OLAP (Online Analytics processing)
- RedShift
#### Elasticache
- In memory caching

### RDS
- Primarily for OLTP (Online Transaction processing)
#### Back ups
    - Automated Backups: Taken daily within a retention period. Also saves transaction logs. So, if restoration is necessary, it can take the latest snapshot and play transaction logs to restore it to any second necessary. Also you get free storage on S3 (for backups) that equals the size of your RDS instance. These are deleted if DB is deleted
    - DB snapshots: These are user initiated. They are retained even if the DB itself is deleted.
- Remember that when a DB is restored, the DNS changes
- Once a DB is encrypted, all copies, replicas, snapshots of that DB will be automatically encrypted
- At present, encrypting an existing DB is not supported. In order to do that, take a snapshot and encrypt the snapshot
- Multi-AZ: AWS maintains an exact replica in different AZs and switches over to the backups when there is a planned maintenance, DB instance failure or AZ failure. **Remember this is for disaster recovery and does not increase performance**. Multi-AZ is enabled by default for Aurora and has to be manually enabled for others
- Read Replicas: These are used for performance improvement from an RDS. These are read-only copies of your production DB. Can be in same AZ, diffenet AZ or even different Region
    - Can enable encryption
    - Must have auto backup enabled
    - Can have upto 5 read replicas for a DB
    - Can have read replicas of read replicas
    - Each read replica has its own DNS
    - Each read replica can have Multi-AZ enabled
    - Can create read replicas of Multi-AZ source DBs
    - Read replicas can be promoted to be their own DB. This stops it from being a replica anymore
    
    
#### Read more on FAQs on RDS
- Microsoft SQL Server default max size is 16TB

### DynamoDB
- Fast, flexible NoSQL with consistent, single-digit millisecond latency at scale. 
- Fully managed DB and supports both document and key-value data models.
- Stored on SSD
- Spread across 3 geographically distinct data centers
- Two read consistency levels
    - Eventual consistency: Consistency is usually reached within a second (best read performance)
    - Strongly consistent read: returns results that reflects all writes that received a successful response prior to the read
- Pricing
    - Write throughput \$0.0065 per hour for every 10 units
    - Read throughtput \$0.0065 per hour for every 50 units
    - storage costs of \$0.25Gb per month
    - One DynamoDB read/write capacity unit can handle 1 read/write per second respectively
    - Scaling on DynamoDB is just a matter of provisioning more read/write capacity that can be done when the Table is up and running!

#### Redshift
- Fast, powerful, fully managed, petabyte-scale data warehouse service in the cloud. 
- Columnar data storage and is great for OLAP
- Advanced Compression techniques as columns are stored together
- Massively Parallel Processing (MPP)
- Customers can start small for just \$0.25 and no commitments and scale up to \$1,000 per TB per year
- Has 2 configurations
    - Single node (160 GB)
    - Multi-Node: Has a **leader node** to manage client connections and **compute nodes** to store data and perform computations (upto 128 Compute nodes)
- Pricing
    - Compute node hours (billed for 1 unit per node per hour) Not charged for leader node hours
    - Charged for backups
    - Charged for data transfer (within VPC)
- Encryption
    - Encrypted in transit using SSL
    - Encrypted at rest using AES-256 encryption
    - By default RedShift manages keys, but user can also manage it
- Availability
    - Currently on available in 1 AZ at a time
    - Can restore snapshots to new AZ if required

### Elasticache
- Web service to deploy, operate and scale an in-memory cache in the cloud
- Two types on AWS
    - Memcached
    - Redis: Has multi-AZ capability

### Aurora
- MySQL compatible
- Five times better perf than MySQL and 1/10th the cost of a commercial DB for comparable perf
- Starts with 10 GB and scales in 10GB increments upto 64TB
- Compute resources can scale upto 32vCPUs and 244 GBs of Memory
- 2 copies of your data is contained in each AZs with a min of 3 AZs. So, 6 copies are maintained
- Aurora is designed to handle loss of upto 2 copies of data w/o affecting write and upto 3 copies w/o affecting read availability
- Storage is self-healing
- 2 Replica types
    - Aurora Replica (upto 15)- for failover, these can take over automatically
    - MySQL read replica (upto 5)


## VPC
- Only one internet gateway allowed per VPC
- 1 subnet is always within 1 AZ
- Consists of IGW or VPGs, Route tables, Network Access Control lists, Subnets and SGs
- Security Groups are Stateful (the inbound connection on a port & protocol will also be allowed to have a response - outbound traffic)
- Network Access Control Lists are stateless (inboud and outbound traffic is to be explicitly permitted)
- VPC peering does not allow transitive connections (A<->B and B<->C does not imply A<->C)
- When a new VPC is created these are auto-generated - SG, NAC, Route table

#### Subnets
- 5 ip address fewer than provisioned. The 1st 4  and last addresses of a CIDR block are reserved by Amazon. 1st one is network address and last one is broadcast address. The other three are reserved by Amazon

#### NAT Instances (depricated)
- Used for servers in a private subnet, there is no internet connection.
- NAT instances must be in a public subnet
- NAT instances are EC2 machines that work as a gateway to internet
- Find AMIs for NAT instances on 3rd party AMIs
- Disable source/destination checks on NAT instances
- Private subnet should have a route to NAT instance for it to be able to access internet

#### NAT Gateway
- Can be provisioned from VPC options. Needs the entry 
- Resides within an AZ
- Scale automatically upto 10Gbps
- Not associated with an SG
- More secure than NAT Instances
- No SSH access to NAT Gateway

#### NAC Lists (Network Access Control) or Network ACL
- One subnet can only be linked to one NAC list
- All VPCs come with a default NACL that allows all inbound and outbound traffic
- By default, if one creates a private NACL, all inbound and outbound traffic is denied
- NACLs are **stateless**
- Can block IP addresses using NACLs and not Security Groups

**Application Load Balancer should be attached to at least 2 Public Subnets**

#### VPC Flow logs
- Cannot enable VPC logging for VPC peers unless the VPC is owned by the same a/c
- Cannot create tags for flow logs
- Cannot change configuration once flow logs are created
- Requests to internal AWS addresses are not logged

#### NAT vs Bastion 
- Bastion lets you SSH into a private server for administration

### Read on
- ENI vs Gateway
- Egress only IG allows IPv6 connection initiated by resources within the VPC
- Check about default inboud and outbound traffic for newly created SG?
- VPC Flow Logs can be created at the VPC, subnet, and network interface levels.

## SQS (Simple Queuing Service)
- Distribute message queueing system
- 256 kB simple text data of any format
- Pull based system
- Messages can be kept in queue from 1 min to 14 days (default is 4 days)
- Long polling is a feature of SQS that allows to notify servers when there is a new message to be processed. This helps to not have the server regularly (short polling) polling SQS looking for messages.
- Retains messages (invisible) for 30 seconds by default after it has been delivered. This is for cases when the processing fails.
    - If process takes longer than 30 secs, increase visibility timeout. Can increase upto 12 hrs.
    - If the process fails, the message can come visible again so that another worker can pick it up
- Two types
    - Simple queues: Guarantees that messages are delivered at least once but does not guarantee order
    - FIFO queues: Guarantees that messages are delivered and also guarantees order. Has a limit of 300 transactions. eg: Bank transactions

## SWF (Simple Workflow Service)
- Three kind of actors - Workflow starter, Deciders, Activity workers
- Ensures that each message is only going to be processed once
- It is defined by 'domain'
- Parameters are specifided in JSON
- Can incorporate human workers too
- Functions in a worker - decider fashion

## Simple Notification Service (SNS)
- Push system
- Publish-subscribe
- Data type is JSON
- Used to send messages triggered from AWS (HTTP, HTTPS, Email, Email-JSON, SQS, Application, Lambda)
- 'Topic' are access points for users to subscribe to identical messages across multiple messaging services
- Pricing 
    - Users pay \$0.5 per 1 million Amazon SNS requests
    - \$0.06 per 100,000 Notification deliveries over HTTP
    - \$0.75 for 100 messages (SMS)
    - \$2 for 100k notifications over email 
    
## Elastic Transcoder
- Converts media (video?) content from one to another format
- Pricing is based on number of minutes transcoded and the resolution

## API Gateway
- Also has a feature called API Caching (has an associated TTL)
- Low cost & Efficient
- Scales effortlessly but have an option to Throttle the gateway to prevent attacks
- Can log results to CloudWatch
- CORS (Cross Origin Resource Sharing): usually browsers only allow content to be read from the same origin for everything in a page - a security feature. CORS relaxes this constraint. *Origin policy cannot be read at the remote resource* is an error from not having set-up CORS
    
## Kinesis
- Used to handle streaming data
- Three services 
    - Streams: Puts data into Shards
        - Retain data from 24 hours to 7 days
        - Capacity of stream is the sum of the capacity of its shards
        - 5 transactions per second for reads, up to max total data read rate of 2 MB per sec and up to 1000 recs per sec for writes, upto a max write rate of 1 MB per sec. 
    - Firehose: Don't have to deal with shards. Can use analytics while data is in Firehose and then write to S3
    - Analytics: Enables you to run SQL queries on data within streams
    